In [1]:
import pandas as pd
import numpy as np

# Importing and merging data

First, let's import and concatenate the Georgia Tech crime logs into a single dataframe and extract and rename relevant columns.

In [2]:
#loading all the csvs into a list
df_list = []

#loading crime data
for year in range(2010, 2019):
    df = pd.read_csv('../Data/{} Crime Log.csv'.format(year))
    #combining IncidentFromDate and IncidentFromTime into a single datetime index
    df['time'] = df['IncidentFromDate'] + ' ' + df['IncidentFromTime']
    df['time'] = pd.to_datetime(df['time'])
    df = df.dropna(subset=['time']).drop_duplicates('time').set_index('time')
    df_list.append(df)
        
#concatenating into single DataFrame
crime_data = pd.concat(df_list).sort_index()

crime_data.head()

,OCANumber,IncidentFromDate,IncidentFromTime,IncidentToDate,IncidentToTime,OffenseCode,Offense Description,CaseStatus,CaseDisposition,LocationCode,PatrolZone,LocationLandmark,LocationStreetNumber,LocationDirectional,LocationStreet,LocationLatitude,LocationLongitude,CreatedSource
time,,,,,,,,,,,,,,,,,,
2010-01-01 01:36:00,10010002,01/01/2010,01:36:00,NaN,NaN,7399,Miscellaneous Offenses,Closed/Cleared,NaN,NONCAM,Z4,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2010-01-01 01:55:00,10010001,01/01/2010,01:55:00,01/01/2010,02:58:00,7399,Miscellaneous Offenses,Closed/Cleared,NaN,ONCAMRES,Z1,Center Street Apartments North 132 Z1,939.0,NaN,"Hemphill Avenue, NW",NaN,NaN,1.0
2010-01-01 02:00:00,10010004,01/01/2010,02:00:00,01/01/2010,04:00:00,2308,Larceny – From Building,Inactive,NaN,NONCAM,Z2,Sigma Nu Fraternity 325 Z2,750.0,NaN,"Fowler Street, NW",NaN,NaN,1.0
2010-01-01 12:00:00,10010032,01/01/2010,12:00:00,01/08/2010,15:20:00,2204,Burglary – No Forced Entry – Residence,Closed/Cleared,Unfounded,ONCAMRES,Z2,North Avenue Apartments North Building,120.0,NaN,"North Avenue, NW",NaN,NaN,1.0
2010-01-02 06:15:00,10010005,01/02/2010,06:15:00,NaN,NaN,7399,Miscellaneous Offenses,Closed/Cleared,NaN,ONCAM,Z2,North Avenue Apartments,120.0,NaN,"North Avenue, NW",NaN,NaN,1.0


In [3]:
#renaming and removing extraneous columns
crime_data = crime_data.iloc[:, [5, 6, 8, 9, 10, 11, 15, 16]]
crime_data.columns = ['code', 'description', 'disposition', 'location', 'patrol_zone', 'landmark', 'lat', 'long']

#adding additional features from time
def set_semester(day_of_year):
    """returns the school semester for a given day of the year"""
    if day_of_year in range(8, 125):
        return 'spring'
    elif day_of_year in range(133, 216):
        return 'summer'
    elif day_of_year in range(232, 349):
        return 'fall'
    else:
        return np.nan

crime_data['year'] = crime_data.index.year
crime_data['month'] = crime_data.index.month
crime_data['day_of_year'] = crime_data.index.dayofyear
crime_data['day_of_month'] = crime_data.index.day
crime_data['day_of_week'] = crime_data.index.dayofweek
crime_data['hour'] = crime_data.index.hour
crime_data['semester'] = crime_data['day_of_year'].apply(set_semester)

crime_data.head()

,code,description,disposition,location,patrol_zone,landmark,lat,long,year,month,day_of_year,day_of_month,day_of_week,hour,semester
time,,,,,,,,,,,,,,,
2010-01-01 01:36:00,7399,Miscellaneous Offenses,NaN,NONCAM,Z4,NaN,NaN,NaN,2010,1,1,1,4,1,NaN
2010-01-01 01:55:00,7399,Miscellaneous Offenses,NaN,ONCAMRES,Z1,Center Street Apartments North 132 Z1,NaN,NaN,2010,1,1,1,4,1,NaN
2010-01-01 02:00:00,2308,Larceny – From Building,NaN,NONCAM,Z2,Sigma Nu Fraternity 325 Z2,NaN,NaN,2010,1,1,1,4,2,NaN
2010-01-01 12:00:00,2204,Burglary – No Forced Entry – Residence,Unfounded,ONCAMRES,Z2,North Avenue Apartments North Building,NaN,NaN,2010,1,1,1,4,12,NaN
2010-01-02 06:15:00,7399,Miscellaneous Offenses,NaN,ONCAM,Z2,North Avenue Apartments,NaN,NaN,2010,1,2,2,5,6,NaN


Now we can import Georgia Tech enrollment data over the years. The data specifies student enrollment segmented by semester from 2010-2018.

In [4]:
gt_enrollment_data = pd.read_csv('../Data/gatech_enrollment.csv')
gt_enrollment_data.head()

,year,semester,enrollment
0,2010,spring,18879
1,2010,summer,9127
2,2010,fall,20726
3,2011,spring,19228
4,2011,summer,9220


In [5]:
gt_enrollment_data['semester'] = gt_enrollment_data['semester']

Now let's import Atlanta demographics data. The columns include information about population by sex and age ranges. They also include information about number of households within certain income brackets and mean and median household incomes.

In [6]:
#importing Atlanta demographics data
atl_data = pd.read_csv('../Data/atlanta_demographics.csv', index_col=0, header=None).transpose()
atl_data.head()

,year,pop_total,pop_male,pop_female,pop_under_5,pop_5_9,pop_10_14,pop_15_19,pop_20_24,pop_25_34,...,inc_15_25,inc_25_35,inc_35_50,inc_50_75,inc_75_100,inc_100_150,inc_150_200,inc_greater_200,inc_median,inc_mean
1,2010,413462,205715,207747,26619,22606,19818,30806,41124,77791,...,18772,18419,21725,26366,16067,18490,8005,14504,45171,79299
2,2011,419250,208201,211049,26903,22296,20093,30602,42382,81418,...,18747,17655,21321,26396,16199,19106,8267,14744,45946,80685
3,2012,425931,211553,214378,26713,22332,20451,29980,44089,83167,...,19410,17754,19900,27102,15527,19099,8380,15080,46146,81292
4,2013,432589,214376,218213,27123,22114,21122,29556,45196,84742,...,19779,17336,19688,26581,16419,19180,8275,15337,46631,82381
5,2014,440641,217938,222703,27976,22336,21576,29071,46287,86918,...,19963,18022,21137,26750,16731,18842,8788,15166,46439,82029


We can now merge the three dataframes together to use for our analysis.

In [7]:
#creating final dataframe
c = crime_data.reset_index()

#merging
c = c.merge(gt_enrollment_data, how='left', on=['year', 'semester'])
c = c.merge(atl_data, how='left', on='year')
c = c.set_index('time')

c.head()

,code,description,disposition,location,patrol_zone,landmark,lat,long,year,month,...,inc_15_25,inc_25_35,inc_35_50,inc_50_75,inc_75_100,inc_100_150,inc_150_200,inc_greater_200,inc_median,inc_mean
time,,,,,,,,,,,,,,,,,,,,,
2010-01-01 01:36:00,7399,Miscellaneous Offenses,NaN,NONCAM,Z4,NaN,NaN,NaN,2010,1,...,18772,18419,21725,26366,16067,18490,8005,14504,45171,79299
2010-01-01 01:55:00,7399,Miscellaneous Offenses,NaN,ONCAMRES,Z1,Center Street Apartments North 132 Z1,NaN,NaN,2010,1,...,18772,18419,21725,26366,16067,18490,8005,14504,45171,79299
2010-01-01 02:00:00,2308,Larceny – From Building,NaN,NONCAM,Z2,Sigma Nu Fraternity 325 Z2,NaN,NaN,2010,1,...,18772,18419,21725,26366,16067,18490,8005,14504,45171,79299
2010-01-01 12:00:00,2204,Burglary – No Forced Entry – Residence,Unfounded,ONCAMRES,Z2,North Avenue Apartments North Building,NaN,NaN,2010,1,...,18772,18419,21725,26366,16067,18490,8005,14504,45171,79299
2010-01-02 06:15:00,7399,Miscellaneous Offenses,NaN,ONCAM,Z2,North Avenue Apartments,NaN,NaN,2010,1,...,18772,18419,21725,26366,16067,18490,8005,14504,45171,79299


# Cleaning columns

In [8]:
c.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 19292 entries, 2010-01-01 01:36:00 to 2018-07-04 20:27:00
Data columns (total 44 columns):
code               19251 non-null object
description        18699 non-null object
disposition        6171 non-null object
location           19241 non-null object
patrol_zone        18874 non-null object
landmark           17992 non-null object
lat                7074 non-null object
long               7078 non-null float64
year               19292 non-null int64
month              19292 non-null int64
day_of_year        19292 non-null int64
day_of_month       19292 non-null int64
day_of_week        19292 non-null int64
hour               19292 non-null int64
semester           17576 non-null object
enrollment         17576 non-null float64
pop_total          19292 non-null int64
pop_male           19292 non-null int64
pop_female         19292 non-null int64
pop_under_5        19292 non-null int64
pop_5_9            19292 non-null int64
pop_10_

Some possible things to look into cleaning:
- lat isn't a float
- code isn't an integer
- many null lat/long coordinates
- lat/long coordinates are sometimes switched
- inconsistent landmark and description naming
- dropping rows in the Savannah campus
- dropping non-crimes

We can start by converting lat to the right type

In [9]:
#filling null values in lat and long with 0
c[['lat', 'long']] = c[['lat', 'long']].fillna(0)


#removing a bad data point, then converting to float
c['lat'] = c['lat'].astype(str)
c = c.loc[c.lat != '`']
c['lat'] = c['lat'].astype(float)

Now we can look into cleaning the code column. Let's look at the values it can take.

In [10]:
c['code'].unique()

array([7399.0, 2308.0, 2204.0, 2305.0, 2404.0, 2317.0, 2902.0, 5707.0,
       3562.0, 2605.0, 2304.0, 4104.0, 2399.0, 2303.0, 5309.0, 5404.0,
       4199.0, 2999.0, 5308.0, 2699.0, 1316.0, 2406.0, 5311.0, 2901.0,
       2203.0, 1315.0, 4902.0, 1313.0, 1399.0, 1199.0, 2314.0, 3564.0,
       4899.0, 1205.0, 2803.0, 4803.0, 3605.0, 1299.0, 2604.0, 2205.0,
       2202.0, 5399.0, 2903.0, 5403.0, 4004.0, 2307.0, 2434.0, 7345.0,
       5299.0, 1103.0, 3599.0, 5314.0, 2408.0, 2502.0, 2301.0, 2302.0,
       1207.0, 9999.0, 5006.0, 4802.0, 3512.0, 3699.0, 3707.0, 5799.0,
       1206.0, 5211.0, 2606.0, 2804.0, 6201.0, 8399.0, 2599.0, 5212.0,
       2701.0, 1203.0, 3502.0, 3532.0, 1204.0, 3611.0, 5215.0, 3571.0,
       '5404', '7399', '9999', '2308', '2399', '5707', '2303', '2304',
       '4104', '1313', '2902', '4199', '2999', '2605', '3562', '3550',
       '2903', '2305', '2901', '1316', '4105', '2699', '2317', '2206',
       '2599', '4802', '5309', '8399', '1399', '2803', '1204', '9999F',
     

Many of these codes are duplicates, only with an extra 0 at the end due to being converted to a float instead of an int.

We can merge them properly by converting to an int, after assigning a unique integer to the codes with alphabetical characters (i.e. 9999W, 9999F, 9999V, 9999T, 9999N, 9999CTW, 9999MH, and 7399R)

In [11]:
#Making a dictionary to map codes with alphabetical values to a unique integer
code_dict = {
    '9999W': '99991',
    '9999F': '99992',
    '9999V': '99993',
    '9999T': '99994',
    '9999N': '99995',
    '9999CTW': '99996',
    '9999MH': '99997',
    '7399R': '73991'
}

#filling NaN values in code so we can convert to int
c['code'] = c['code'].fillna(0)
#Mapping only the records with codes in the dictionary
c.loc[c.code.isin(code_dict.keys()), 'code'] = c['code'].map(code_dict)
#converting to int
c['code'] = c['code'].astype(int)

c['code'].unique()

array([ 7399,  2308,  2204,  2305,  2404,  2317,  2902,  5707,  3562,
        2605,  2304,  4104,  2399,  2303,  5309,  5404,  4199,  2999,
        5308,  2699,  1316,  2406,  5311,  2901,  2203,  1315,  4902,
        1313,  1399,  1199,  2314,  3564,  4899,  1205,  2803,  4803,
        3605,  1299,  2604,  2205,  2202,  5399,  2903,  5403,  4004,
        2307,  2434,  7345,  5299,  1103,  3599,  5314,  2408,  2502,
        2301,  2302,  1207,  9999,  5006,  4802,  3512,  3699,  3707,
        5799,  1206,  5211,  2606,  2804,  6201,  8399,  2599,  5212,
        2701,  1203,  3502,  3532,  1204,  3611,  5215,  3571,  3550,
        4105,  2206, 99992,  5499,     0,  2424, 99991,  2099,  2589,
        2602,  5203,  1102,  3572,  4101,  5202,  3561,  7901,  2501,
        5312,  2199,  5213,  3513,  3504,  3805,  6399,  3899,  1314,
        3542,  3522,  3799,  1209,  3560,  1104,  4801,  3533, 99994,
       99995, 73991,  2310, 99993,  1116,  5313, 99996,  1099, 99997],
      dtype=int64)

Now that the code column is clean, we can look into filling some of the empty coordinates. I will fill rows that have a landmark but no coordinate data with the coordinate data from other rows with the same landmark.

Let's look at the landmark column first.

In [12]:
c['landmark'].value_counts().head(10)

Beringause Building   46   Z1          433
North Avenue Apartments                421
Campus Recreation Center               314
Beringause Building                    298
Student Center                         298
Library   77   Z2                      227
Student Center   104   Z3              206
Campus Recreation Center   122   Z4    178
Barnes and Noble Bookstore             152
Library                                151
Name: landmark, dtype: int64

Many of the landmarks are the same, but some have the associated zone patrol info added to the end. Let's get rid of this zone patrol info to merge the values with the same landmark. Notice how the values with the added zone info have the pattern:

some spaces - a number - some spaces - zone

We can look for this pattern with regular expression. 

In [13]:
#filling null values with blank strings
c['landmark'] = c['landmark'].fillna('')
#truncating the matched pattern from values with zone info
c['landmark'] = c['landmark'].str.replace("\s+\d+\s+Z\d", '')

c['landmark'].value_counts().head(10)

                                        1300
Beringause Building                      732
Campus Recreation Center                 559
North Avenue Apartments                  525
Student Center                           506
Library                                  378
Barnes and Noble Bookstore               268
Bobby Dodd Stadium                       239
Clough Undergraduate Learning Center     204
Student Health Center                    188
Name: landmark, dtype: int64

This data looks much more clean. Now we can use these landmarks to fill in missing lat/long coordinates. First we'll need to swap any lat/long coordinates that were entered in reverse.

In [14]:
def swap_bad_coords(row):
    """takes in a row of c and swaps the latitude and longitude
    values if they were entered in reverse"""
    if row['lat']<0 or row['long']>0:
        return [row['long'], row['lat']]
    else:
        return [row['lat'], row['long']]
    
c.loc[:, ['lat', 'long']] = c.loc[:, ['lat', 'long']].apply(swap_bad_coords, axis='columns')

Now that the coordinates are valid, we can fill in the missing ones.

In [15]:
#creating a dataframe that links landmarks to their coordinates
landmark_coord_df = c[['landmark', 'lat', 'long']][(c.lat != 0)&(c.long != 0)]
landmark_coord_df = landmark_coord_df.drop_duplicates('landmark').set_index('landmark')

landmark_lat = landmark_coord_df['lat']
landmark_long = landmark_coord_df['long']

    
landmark_coord_df.head()

,lat,long
landmark,,
Technology Square Parking Deck,33.775559,-84.388811
Beringause Building,33.779417,-84.402153
Global Learning Center,33.776862,-84.389938
"Hemphill Avenue, NW @ Tenth Street, NW",33.772482,-84.404264
Arby's,33.763993,-84.388985


In [16]:
print("Number of missing latitudes before:", c[c.lat == 0].shape[0])
print("Number of missing longitudes before", c[c.long == 0].shape[0])

#filling missing values by mapping from appropriate series
c.loc[c.landmark.isin(landmark_coord_df.index), 'lat'] = c.landmark.map(landmark_lat)
c.loc[c.landmark.isin(landmark_coord_df.index), 'long'] = c.landmark.map(landmark_long)

print("Number of missing latitudes after:", c[c.lat == 0].shape[0])
print("Number of missing longitudes after:", c[c.long == 0].shape[0])

Number of missing latitudes before: 12218
Number of missing longitudes before 12213
Number of missing latitudes after: 3694
Number of missing longitudes after: 3692


The coordinate columns are now clean, so let's look at the location column.

In [17]:
c['location'].value_counts(dropna=False)

ONCAM       10673
ONCAMRES     3430
PUB          3234
NONCAM       1260
NONCLERY      641
NaN            51
oncam           1
oncamres        1
Name: location, dtype: int64

The missing values here aren't as important, so we don't need to worry about filling them in. But we will want to capitalize all the values to ensure consistent naming.

In [18]:
#capitalizing all entries in location
c['location'] = c['location'].str.upper()
c['location'].value_counts(dropna=False)

ONCAM       10674
ONCAMRES     3431
PUB          3234
NONCAM       1260
NONCLERY      641
NaN            51
Name: location, dtype: int64

Now we can fix errors in the description column. Many of the crime descriptions are written with a en dash (–) and many are written with a hyphen (-). Replacing all the en dashes with hyphens will help later.

In [19]:
c['description'] = c['description'].str.replace('–', '-')

Finally we need to drop any rows we can't use for our analysis. We will want to drop any rows that don't have any useful location or description data, as well as any records that aren't crimes. These non-crimes are represented by the code 9999.

In [20]:
print("Shape before dropping any records:", c.shape)

#dropping crimes without any location information
c = c[(c.landmark!='') | ((c.lat!=0) & (c.long!=0))]
#dropping crimes at the Savannah campus
c = c[c.patrol_zone != 'SAV']
print("Shape after dropping crimes without locations:", c.shape)


#dropping records without any descriptive information
c = c[c.code != 0]
print("Shape after dropping crimes without code:", c.shape)

#dropping non=crimes
c = c[c.code != 9999]
print("Shape after dropping non-crimes:", c.shape)

Shape before dropping any records: (19291, 44)
Shape after dropping crimes without locations: (19283, 44)
Shape after dropping crimes without code: (19242, 44)
Shape after dropping non-crimes: (12486, 44)


In [21]:
c.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12486 entries, 2010-01-01 01:36:00 to 2018-07-04 01:56:00
Data columns (total 44 columns):
code               12486 non-null int32
description        11934 non-null object
disposition        5962 non-null object
location           12465 non-null object
patrol_zone        12212 non-null object
landmark           12486 non-null object
lat                12486 non-null float64
long               12486 non-null float64
year               12486 non-null int64
month              12486 non-null int64
day_of_year        12486 non-null int64
day_of_month       12486 non-null int64
day_of_week        12486 non-null int64
hour               12486 non-null int64
semester           11335 non-null object
enrollment         11335 non-null float64
pop_total          12486 non-null int64
pop_male           12486 non-null int64
pop_female         12486 non-null int64
pop_under_5        12486 non-null int64
pop_5_9            12486 non-null int64
pop_1

The data is now in a much more standardized and clean form, and is ready to be used for analysis.

In [22]:
#writing clean data to new csv
c.to_csv('../Data/crime_data_clean.csv', encoding='utf-8')